# Vehicle Detection
### by Kilian Muelken

---
#### Code Cell 1: necessary imports for the code

In [1]:
import numpy as np
import cv2
import glob
import time
from skimage.feature import hog
from moviepy.editor import VideoFileClip
from sklearn.cross_validation import train_test_split
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.externals import joblib
import copy
from scipy.ndimage.measurements import label
from random import randint
from random import shuffle
from collections import deque

___
## Part 1: Vehicle detection functions
*Functions for the vehicle Detection*

### Part 1.1: Common functions

___
#### Code Cell ?: Plot and save images
**Input:** list of images<br>
**Output:** plots and saves images

In [2]:
def plotImage(imgs, save=False):
    # check if imgs is a list
    imglist = []
    
    if isinstance(imgs, list):
        imglist = imgs
    else:
        imglist.append(imgs)
        
    n_img = len(imglist)
    
    plt.figure(figsize=(20,5))
    
    for i in range(n_img):
        # plot image
        n_plot = 101 + n_img*10 + i
        plt.subplot(n_plot) # source: https://stackoverflow.com/questions/35692507/plot-several-image-files-in-matplotlib-subplots
        plt.imshow(imglist[i])
        
        # save image if save=true
        if save:
            #img_BGR = cv2.cvtColor(img_list[i],cv2.COLOR_RGB2BGR)
            img_name = 'image_'+time.strftime('%m_%Y_%H_%M_%S')+'.jpg'
            mpimg.imsave('output_images/'+img_name, imglist[i])

___
#### Code Cell ?: Draw bounding boxes
**Input:** image to draw the boxes, position and size of the boxes<br>
**Output:** image with boxes drawn

In [3]:
def drawBoxes(img, bboxes, color = (0, 0, 255), thick = 6):
    #color = (0, 0, 255) # Color of the boxes is blue
    #thick = 6  # thickness of the boxes
    
    img_boxes = np.copy(img) # Make a copy of the image
    
    for bbox in bboxes: # Iterate through the bounding boxes
        cv2.rectangle(img_boxes, bbox[0], bbox[1], color, thick) # Draw a rectangle given bbox coordinates
    
    return img_boxes # return the image with boxes drawn

#### Code Cell ?: Read test images
**Input:** path of the test images<br>
**Output:** images with marked detected vehicles

In [4]:
def readImages(path):
    images = []
    images_list = glob.glob(path) # Make a list of calibration images

    # Step through the list, read test images and convert them to RGB
    for fname in images_list:
        img = cv2.imread(fname)
        rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        images.append(rgb)
    return images

___
#### Code Cell ?: Find and merge mulitple detections
**Input:** Detected windows with cars<br>
**Output:** windows with merged multiple detections

In [5]:
def getbboxes(labels):
    bboxes = []
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        
        bboxes.append(bbox)

    # Return the bboxes
    return bboxes


        
def multipleDetections(image, windows):
    global heatmaps
    threshold = 10
    
    new_heatmap = np.zeros_like(image[:,:,0]).astype(np.float)
     
    # add new heat to the map
    for box in windows:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        new_heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1
        
    # add old heatmaps
    heatmaps.append(new_heatmap)# append new heatmap
      
    for old_heatmap in heatmaps:
        new_heatmap = (new_heatmap*len(heatmaps)+old_heatmap)/(len(heatmaps)+1)
            
    if len(heatmaps) > 10:
        heatmaps.popleft()
            
    print(new_heatmap.max())
    
    # reduce heat less than threshold
    new_heatmap[new_heatmap <= threshold] = 0
    
    # detect cars on image
    labels = label(new_heatmap)
    
    # get boxes around the detected cars
    windows_final = getbboxes(labels)
    
    # draw boxes on the image
    image_final = drawBoxes(image, windows_final)
    
    return image_final

___
### Part 1.2: Feature functions

___
#### Code Cell ?: Compute HOG features
**Input:** Image<br>
**Output:** HOG features

In [6]:
def hogFeatures(img):
    orientations = 9
    pix_per_cell = 8
    pixels_per_cell=(pix_per_cell, pix_per_cell)
    cell_per_block = 2
    cells_per_block=(cell_per_block, cell_per_block)
    visualise = False
    feature_vector = False # was True!!
    hog_channel = 'ALL'
    
    if hog_channel == 'ALL':
        features = []
        for channel in range(img.shape[2]):
            features_channel = hog(img[:,:,channel], orientations=orientations, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       visualise=visualise, feature_vector=feature_vector)
            features.append(features_channel)
        features = np.ravel(features)
    else:
        img_hog = img[:,:,hog_channel]
        features = hog(img_hog, orientations=orientations, 
                    pixels_per_cell=(pix_per_cell, pix_per_cell),
                    cells_per_block=(cell_per_block, cell_per_block), 
                    block_norm= 'L2-Hys',
                    transform_sqrt=True, 
                    visualise=visualise, feature_vector=feature_vector)
    return features

___
#### Code Cell ?: Compute binned color features 
**Input:** Image<br>
**Output:** binned color features

In [7]:
def binSpatial(img):
    size=(32, 32)
    
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    
    # Return the feature vector
    return features

___
#### Code Cell ?: Compute color histogram features 
**Input:** Image<br>
**Output:** binned color features

In [8]:
def colorHist(img):
    nbins=32
    bins_range=(0, 256) # NEED TO CHANGE bins_range if reading .png files with mpimg!
        
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    
    # Concatenate the histograms into a single feature vector
    features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    
    # Return the individual histograms, bin_centers and feature vector
    return features

___
#### Code Cell ?: Extract features 
*Uses hogFeatures(), binSpatial() and colorHist()*
**Input:** Image<br>
**Output:** features

In [9]:
def extractFeatures(imgs):
    color_space = 'YCrCb'
    spatial_feat = True
    hist_feat  =True
    hog_feat = True
    
    # check if imgs is a list
    if isinstance(imgs, list):
        imglist = imgs
    else:
        imglist = []
        imglist.append(imgs)
        
    # Create a list to append feature vectors to
    features = []
    
    for img in imglist:
        img_features = []
    
        # color conversion if other than 'RGB'
        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
            elif color_space == 'HLS':
                    feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
        else:
            feature_image = np.copy(img) 

        # Compute binned color features
        if spatial_feat == True:
            spatial_features = binSpatial(feature_image)
            img_features.append(spatial_features)
            

        # Compute color histogram features
        if hist_feat == True:
            hist_features = colorHist(feature_image)
            img_features.append(hist_features)
            

        # Compute HOG features
        if hog_feat == True:
            hog_features = hogFeatures(feature_image)
            img_features.append(hog_features)

        features.append(np.concatenate(img_features))
        
    # Return list of feature vectors
    return features

___
### Part 1.3: Window functions

___
#### Code Cell ?: Calculate sliding windows
**Input:** image, start and stop positions in both x and y, window size (x and y dimensions),  and overlap fraction (for both x and y)<br>
**Output:** List of sliding windows

In [10]:
def calcSlideWindows(img, xy_window, xy_overlap, x_start, x_stop, y_start, y_stop):
        
    # Compute the span of the region to be searched    
    xspan = x_stop - x_start
    yspan = y_stop - y_start
    
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    
    # Initialize a list to append window positions to
    slidingWindows = []
    
    # Loop through finding x and y window positions
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start
            endy = starty + xy_window[1]
            
            # Append window position to list
            slidingWindows.append(((startx, starty), (endx, endy)))
    
    # Return the list of windows
    return slidingWindows

___
#### Code Cell ?: Add sliding windows
**Input:** window with positive detection, list with all sliding windows<br>
**Output:** new sliding windows

In [11]:
def addSlideWindows(window, windows):
    sorround_windows = []
    diff = 2
    if window[0][0]-diff >= 0:
        sorround_windows.append(((window[0][0]-diff, window[0][1]), (window[1][0]-diff, window[1][1]))) # left window
    if window[1][0]+diff <= 1280:
        sorround_windows.append(((window[0][0]+diff, window[0][1]), (window[1][0]+diff, window[1][1]))) # right window
    
    new_windows = []
    for sorround_window in sorround_windows:
        if sorround_window not in windows:
            new_windows.append(sorround_window)
        
    return new_windows

___
#### Code Cell ?: Search for positive matches in windows
*Uses extractFeatures()*
**Input:** Image, Silde windows, Classifier, Scaler<br>
**Output:** positive matches in windows

In [12]:
def searchPositiveWindows(img, windows, clf, scaler):
    
    # Create an empty list to receive positive detection windows
    on_windows = []
    
    # Iterate over all windows in the list
    for window in windows:
        # Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))
        
        # Extract features for that window using single_img_features()
        features = extractFeatures(test_img)
        
        # Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        
        # Predict using your classifier
        prediction = clf.predict(test_features)
        
        # If positive (prediction == 1) then save the window
        if prediction == 1:
            add_windows = addSlideWindows(window, windows)
            windows.extend(add_windows)
            on_windows.append(window)
            
    # Return windows for positive detections
    return on_windows

___
## Part ?: Training of the Classifier
*Training of the Classifier with test images*<br>

#### Code Cell ?: Training Pipeline
**Input:** path of the training images<br>
**Output:** trained classifier

In [13]:
def trainingPipeline(cars, notcars):
    n_cars = len(cars)
    n_notcars = len(notcars)
    
    if n_cars < n_notcars:
        notcars = notcars[0:n_cars]
    elif n_notcars < n_cars:
        cars = cars[0:n_notcars]
    
    car_features = extractFeatures(cars)
    notcar_features = extractFeatures(notcars)
        
    # Create an array stack of feature vectors
    X = np.vstack((car_features, notcar_features)).astype(np.float64)

    # Define the labels vector
    y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

    # Split up data into randomized training and test sets
    rand_state = np.random.randint(0, 100)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=rand_state)

    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X_train)
    
    # Apply the scaler to X
    X_train = X_scaler.transform(X_train)
    X_test = X_scaler.transform(X_test)
    
    # Use a linear SVC 
    svc = LinearSVC()
    svc.fit(X_train, y_train)
    
    # Check the score of the SVC
    print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
    
    return svc, X_scaler

In [14]:
training = False # Set to True for new training

if training:
    # read train images
    notcars = readImages('train_nv/*.png')
    cars = readImages('train_v/*.png')

    # calculate classifier and scaler
    clf, scaler = trainingPipeline(cars, notcars)

    # save the classifier and scaler to the disk
    # source: http://scikit-learn.org/stable/modules/model_persistence.html
    joblib.dump(clf, 'clf.pkl')
    joblib.dump(scaler, 'scaler.pkl')

In [15]:
# load the classifier and scaler
# source: http://scikit-learn.org/stable/modules/model_persistence.html
clf = joblib.load('clf.pkl')
scaler = joblib.load('scaler.pkl')

___
## Part ?: Testing
*Test of the Classifier with test images and test video*<br>

___
#### Code Cell ?: Image Testing Pipeline
*Test of the Classifier with test images*<br>
**Input:** path of the test images<br>
**Output:** images with marked detected vehicles

In [16]:
def pipeline(img):    
    global car_windows
    image = np.copy(img) # Make a copy of the image
    
    # calculate the searching windows
    slide_windows = calcSlideWindows(image, (32, 32), (0.5, 0.5), 320, 960, 400, 464)
    slide_windows.extend(calcSlideWindows(image, (64, 64), (0.5, 0.5), 128, 1152, 400, 464))
    slide_windows.extend(calcSlideWindows(image, (96, 96), (0.5, 0.5), 96, 1184, 400, 496))
    slide_windows.extend(calcSlideWindows(image, (128, 128), (0.5, 0.5), 0, 1280, 400, 528))
    slide_windows.extend(calcSlideWindows(image, (128, 128), (0.5, 0.5), 1088, 1280, 350, 478))
    slide_windows.extend(calcSlideWindows(image, (160, 160), (0.5, 0.5), 0, 1280, 400, 560))
    slide_windows.extend(calcSlideWindows(image, (192, 192), (0.5, 0.5), 16, 1264, 400, 592))
    slide_windows.extend(calcSlideWindows(image, (256, 256), (0.5, 0.5), 640, 1280, 400, 656))
    
    slide_windows.extend(car_windows)
    
    # find positive matches in searching windows
    car_windows = searchPositiveWindows(image, slide_windows, clf, scaler)
    #image = drawBoxes(image, car_windows, color = (0, 255, 0), thick = 1)

    # merge multiple detections
    image_cars = multipleDetections(image, car_windows)
        
    return image_cars

In [17]:
# read test images
if False:
    test_images = readImages('test_images/*.jpg')

    images = []

    for test_image in test_images:
        # pipeline
        test_image_detected = pipeline(test_image)
        images.append(test_image_detected)

    plotImage(images)

In [18]:
#plotImage(images[5])

#### Code Cell ?: Detect vehicles on video
**Input:** name of the test video<br>
**Output:** video with marked detected vehicles will be written in current folder

In [19]:
def detectVehicles(video_file_name):
    global heatmaps # detected heatmaps from the last images
    heatmaps = deque() # source: https://docs.python.org/2/library/collections.html
    global car_windows
    car_windows = []
    #clip = VideoFileClip(video_file_name+'.mp4')
    clip = VideoFileClip(video_file_name+'.mp4').subclip(23,30)
    video_clip = clip.fl_image(pipeline)
    %time video_clip.write_videofile(video_file_name+'_with_detected_vehicles.mp4', audio=False)

In [20]:
detectVehicles('project_video')

22.0
[MoviePy] >>>> Building video project_video_with_detected_vehicles.mp4
[MoviePy] Writing video project_video_with_detected_vehicles.mp4


  0%|                                                                                          | 0/176 [00:00<?, ?it/s]

22.7777777778


  1%|▍                                                                                 | 1/176 [00:01<04:02,  1.38s/it]

23.53125


...

345.827673518


 99%|███████████████████████████████████████████████████████████████████████████████ | 174/176 [05:00<00:07,  3.53s/it]

358.193258489


 99%|███████████████████████████████████████████████████████████████████████████████▌| 175/176 [05:04<00:03,  3.65s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_with_detected_vehicles.mp4 

Wall time: 5min 5s
